## Librerías

In [10]:
%run "../recurrentes.ipynb"

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [11]:
%run "../funciones.ipynb"

In [12]:
%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False
%sql duckdb:///:memory:


The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [13]:
#1 - Carga del dataset. Opciones: 
# dataset_lags_clase_ternaria_l : dataset con lags y clase ternaria
# dataset_clase_ternaria_l: dataset sin lags y clase ternaria

data = pd.read_parquet(dataset_clase_ternaria_l)

## Valores null por variable

In [14]:
# Ingresar mes train y mes score
train_data = data[data['foto_mes'] == 202108]
score_data = data[data['foto_mes'] == 202106]

train_null_percentage = train_data.isnull().mean() * 100
score_null_percentage = score_data.isnull().mean() * 100

comparison_df = pd.DataFrame({'Train Null Percentage': train_null_percentage, 'Score Null Percentage': score_null_percentage})
comparison_df['diff'] = (comparison_df['Score Null Percentage'] - comparison_df['Train Null Percentage']).abs()

comparison_df_sorted = comparison_df.sort_values('diff', ascending=False)

comparison_df_sorted

,Train Null Percentage,Score Null Percentage,diff
Visa_Finiciomora,97.495195,99.390451,1.895256
mtarjeta_master_descuentos,3.453778,2.682016,0.771763
Visa_mconsumosdolares,12.844381,13.081952,0.237572
Visa_cadelantosefectivo,12.844381,13.081952,0.237572
Visa_cconsumos,12.844381,13.081952,0.237572
...,...,...,...
mpayroll,0.000000,0.000000,0.000000
mpayroll2,0.000000,0.000000,0.000000
cpayroll2_trx,0.000000,0.000000,0.000000
ccuenta_debitos_automaticos,0.000000,0.000000,0.000000


## Análisis drifting con PSI

In [15]:
psi_results = []
for column in train_data.columns:
  if column not in ['foto_mes', 'clase_ternaria']:
    train_variable = train_data[column]
    score_variable = score_data[column]
    psi_value = psi(train_variable, score_variable)
    psi_results.append({'feature': column, 'psi': psi_value})

psi_df = pd.DataFrame(psi_results)
psi_df = psi_df.sort_values('psi', ascending=False)
psi_df

,feature,psi
135,Visa_Finiciomora,2.288712
122,Master_fultimo_cierre,1.129652
144,Visa_fultimo_cierre,0.937082
50,cpayroll_trx,0.223737
145,Visa_mpagado,0.136970
...,...,...
40,cinversion1,0.000000
39,mplazo_fijo_pesos,0.000000
95,thomebanking,0.000000
93,tcallcenter,0.000000


In [16]:
variable_name = 'Master_Finiciomora'
expected = train_data[variable_name]
actual = score_data[variable_name]

expected_not_null = expected.dropna()
actual_not_null = actual.dropna()

bin_edges = pd.qcut(expected_not_null, q=10, duplicates='drop').unique()
bin_edges2 = [edge.left for edge in bin_edges] + [edge.right for edge in bin_edges]
breakpoints = sorted(list(set(bin_edges2)))

print(f'Cortes en {variable_name}: {breakpoints}')
expected_counts, _ = np.histogram(expected_not_null, bins=breakpoints)
actual_counts, _ = np.histogram(actual_not_null, bins=breakpoints)

print(f'Frecuencia Esperada: {expected_counts}')
print(f'Frecuencia Actual: {actual_counts}')


Cortes en Master_Finiciomora: [5.999, 15.0, 22.0, 29.0, 50.0, 57.0, 239.0]
Frecuencia Esperada: [ 27  59 313  46  94  66]
Frecuencia Actual: [ 41  25 282  49 117 119]


## Analisis drifting con modelo

In [19]:
# 2- Para el caso de que el dataset sea con feature engineering incluido
if 'T_visa_normal' in data.columns:
    data['T_visa_normal'] = data['T_visa_normal'].astype(bool)
    data['T_master_normal'] = data['T_master_normal'].astype(bool)


In [21]:
# 3- parametros para calcular ganancia del modelo
ganancia_acierto = 273000
costo_estimulo = 7000


data['clase_peso'] = 1.0
data.loc[data['clase_ternaria'] == 'BAJA+2', 'clase_peso'] = 1.00002
data.loc[data['clase_ternaria'] == 'BAJA+1', 'clase_peso'] = 1.00001


In [22]:
# 4 - binarización de la clase ternaria

data['clase_binaria'] = np.where(data['clase_ternaria']=='CONTINUA', 0, 1)

In [23]:
# 5 Especificar mes de train y test

df_train = data[data['foto_mes']<=202106]
df_test = data[data['foto_mes']==202108]


In [24]:
clase_peso = df_train['clase_peso']
X_train = df_train.drop(['clase_ternaria', 'clase_binaria', 'clase_peso'], axis=1)
Y_train =df_train['clase_binaria']
X_test = df_test.drop(['clase_ternaria', 'clase_binaria', 'clase_peso'], axis=1)
Y_test =df_test['clase_binaria']

In [25]:
w_train = df_train.loc[X_train.index, 'clase_peso']


In [26]:
best_params = {'num_leaves': 40,
 'learning_rate': 0.022596031021514126,
 'min_data_in_leaf': 1692,
 'feature_fraction': 0.8060065372089812,
 'bagging_fraction': 0.8663914349386255}

In [ ]:
train_data = lgb.Dataset(X_train,
                            label=Y_train,
                            weight=w_train)


params = {
'objective': 'binary',
'boosting_type': 'gbdt',
'first_metric_only': True,
'boost_from_average': True,
'feature_pre_filter': False,
'max_bin': 31,
'num_leaves': 0,
'learning_rate': 0,
'min_data_in_leaf': 0,
'feature_fraction': 0,
'bagging_fraction': 0,
'seed': semillas[0],
'verbose': 0
}
    #actualización con valores optuna
params.update(best_params)

if params['num_leaves'] == 0:
    print ('Error falta actualización de parámetros')


model = lgb.train(params,
                train_data,
                num_boost_round=299)



In [ ]:
model_df = model.trees_to_dataframe()

In [ ]:
model_df[model_df['split_feature'] == 'cpayroll_trx'].sort_values('threshold')

## Analisis SHAPs

In [ ]:
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_train)

##